In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# K-Means Clustering: A Larger Example

more than 2D

show cross validation for choosing k with elbow plot

multiple runs with different initializations and choose lowest loss

interesting dataset? gene expression?

## Disadvantages of `K-Means` Clustering

As we discussed above, k-means clustering has several disadvantages. It does not always converge to a solution that provides the global minimum within-cluster variability. Because of this, it can also give differing solutions depending on the initial starting points. In addition, the k-means algorithm requires the user to specify the number of clusters, which may not always be obvious, especially for data with high dimensionality. In the next section, we will discuss another clustering method that does not require you to specify a number of clusters: hierarchical clustering.